# 1. Dataset

In [4]:
# DATASET
import torch
import torch.nn as nn
import torchvision, torchinfo, torchmetrics
import torchvision
from sklearn.model_selection import train_test_split
import os, glob, zipfile
from tqdm import tqdm
from PIL import Image

class Custom_Dataset(torch.utils.data.Dataset):
    def __init__(self,file_list):
        self.file_list = file_list
        self.transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize((224, 224)), # IMPORTANT SIZE OF IMAGE (224, 224) or (256,256)
                torchvision.transforms.ToTensor(),
            ])

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self,idx):
        img_path        = self.file_list[idx]
        img             = Image.open(img_path)
        img_transformed = self.transform(img)

        label = img_path.split('/')[-1].split('.')[0]
        if label == 'dog':
            label=1
        elif label == 'cat':
            label=0

        return img_transformed,label

# zip_files = ['train.zip', 'test.zip']
# for zip_file in zip_files:
#     with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/{}".format(zip_file),"r") as z:
#         z.extractall(".")
#         print("{} unzipped".format(zip_file))

train_file_names_list = glob.glob(os.path.join("../working/train",'*.jpg'))
test_file_names_list  = glob.glob(os.path.join("../working/test", '*.jpg'))

train_list, val_list  = train_test_split(train_file_names_list, test_size=0.2)

train_dataset = Custom_Dataset(train_list)
val_dataset   = Custom_Dataset(val_list)

training_dataloader    = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=32, shuffle=True )
validation_dataloader  = torch.utils.data.DataLoader(dataset = val_dataset  , batch_size=32, shuffle=False)

assert next(iter(training_dataloader))   is not None
assert next(iter(validation_dataloader)) is not None

# 2. Model Training

In [5]:
# TRAINING
import lightning
import torchmetrics

class Lightning_Module(lightning.LightningModule):
    def __init__(self, model):
        super().__init__()
        
        self.model = model
        self.automatic_optimization = False
        self.training_accuracy   = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.validation_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10)
  
    def training_step (self, batch, batch_idx):
        images_actual, labels_actual        = batch
        predicted_logits  = self.model(images_actual)
        labels_predicted  = torch.argmax(predicted_logits, dim = 1)

        loss              = torch.nn.functional.cross_entropy(predicted_logits, labels_actual)
        
        optimizer         = self.optimizers()
        optimizer.zero_grad()
        self.manual_backward(loss)
        optimizer.step()
        """
        for individual_parameter in self.parameters():
            individual_parameter = individual_parameter - individual_parameter.grad * learning_rate
        """
        self.training_accuracy(labels_predicted, labels_actual)
        self.log("train_loss"     , loss                   , prog_bar = True)
        self.log("train_accuracy" , self.training_accuracy , prog_bar = True)
        
        return loss

    def validation_step (self, batch, batch_idx):
        images_actual, labels_actual        = batch
        predicted_logits  = self.model(images_actual)
        labels_predicted  = torch.argmax(predicted_logits, dim = 1)

        loss = torch.nn.functional.cross_entropy(predicted_logits, labels_actual)
        self.validation_accuracy(labels_predicted, labels_actual)
        self.log("validation_loss"     , loss                     , prog_bar= False)
        self.log("validation_accuracy" , self.validation_accuracy , prog_bar= True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer

# 3. Model Architecture

In [6]:
# MODEL ARCHITECTURE
feature_extractor = nn.Sequential(
    # Standard Input Size: 3, 224, 224
    # Filter: UV Light filter on glasses is looking for UV lights to filter out or Filter in water purification, is looking for impurities
    nn.Conv2d                           ( in_channels =  3, out_channels = 50, kernel_size = (3,3), padding="same"), 
    nn.ReLU(), nn.MaxPool2d ((2,2), 2),                                           
    # 112, 112
    nn.Conv2d                           ( in_channels = 50, out_channels = 50, kernel_size = (3,3), padding="same"),
    nn.ReLU(), nn.MaxPool2d ((2,2), 2),
    # 56, 56
    nn.Conv2d                           ( in_channels = 50, out_channels = 50, kernel_size = (3,3), padding="same"),
    nn.ReLU(), nn.MaxPool2d ((2,2), 2),
    # 28, 28
    nn.Conv2d                           ( in_channels = 50, out_channels = 50, kernel_size = (3,3), padding="same"),
    nn.ReLU(), nn.MaxPool2d ((2,2), 2),
    # 14, 14
    nn.Conv2d                           ( in_channels = 50, out_channels = 512, kernel_size = (3,3), padding="same"),
    nn.ReLU(), nn.MaxPool2d ((2,2), 2),
    # Standard Output Size: 512, 7, 7
    # Total Features = 512 images of 7*7 pixel
)

decision_maker = nn.Sequential(
    nn.Flatten(start_dim=1),
    nn.Linear(in_features = 512*7*7 , out_features = 50), nn.ReLU(),
    nn.Linear(in_features = 50      , out_features = 2) # 1 Neuron is for detecting Cat and 2nd Neuron is for detecting Dog
)

model = nn.Sequential(
  feature_extractor,
  decision_maker
)

# 4. Training Model

In [7]:
epochs  = 10
lightning_model   = Lightning_Module(model)
lightning_trainer = lightning.Trainer( max_epochs= epochs, log_every_n_steps= 50)

lightning_trainer.fit(model=lightning_model, train_dataloaders= training_dataloader, val_dataloaders= validation_dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/ajinkyak/Codes/Cats vs Dogs/lightning_logs

  | Name                | Type               | Params | Mode 
-------------------------------------------------------------------
0 | model               | Sequential         | 1.6 M  | train
1 | training_accuracy   | MulticlassAccuracy | 0      | train
2 | validation_accuracy | MulticlassAccuracy | 0      | train
-------------------------------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.218     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:475: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


# 5. Save Model

In [ ]:
torch.save(model.state_dict(), 'trained_model_parameter_values.pth')

# Load a trained model

model.load_state_dict(torch.load(filename))

In [ ]:
!tensorboard --logdir=./lightning_logs 